In [5]:
!pip install FlagEmbedding

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached multidict-6.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.0 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 511.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 865.7 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 411.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
#Import Modules
import pickle
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModelForCausalLM
from FlagEmbedding import FlagModel
from numpy.linalg import norm

ImportError: cannot import name 'GenerationMixin' from 'transformers.generation' (/opt/homebrew/anaconda3/envs/urbandesignpython/lib/python3.12/site-packages/transformers/generation/__init__.py)

In [ ]:
# Download NLTK data
nltk.download('punkt_tab')

In [3]:
# Load the combined database from the pickle file
pickle_folder = "./pickle_file"
database = []

for pickle_file in os.listdir(pickle_folder):
    if pickle_file.endswith('.pkl'):
        with open(os.path.join(pickle_folder, pickle_file), "rb") as f:
            database.extend(pickle.load(f))

In [4]:
# Tokenize sentences for Word2Vec
tokenized_sentences = [word_tokenize(entry['Processed Text'].lower()) for entry in database]

In [5]:
# Save tokenized sentences
with open('tokenized_sentences.pickle', 'wb') as gerkin:
    pickle.dump(tokenized_sentences, gerkin)

In [6]:
# Load tokenized sentences
with open('tokenized_sentences.pickle', 'rb') as gerkin:
    tokenized_sentences = pickle.load(gerkin)

In [7]:
# Train Word2Vec Model
w2v_model = Word2Vec(sentences=tokenized_sentences, vector_size=50, window=3, min_count=5, workers=4)
w2v_model.save("word2vec.model")

In [8]:
# Debug: Check the vocabulary size of the Word2Vec model 
print("Vocabulary size of the Word2Vec model:",len(w2v_model.wv))

Vocabulary size of the Word2Vec model: 6288


In [10]:
# Print words from-to index and their vectors from the Word2Vec model's vocabulary
start_index = 100
end_index = 200
vocab = list(w2v_model.wv.key_to_index.items())[start_index:end_index]
print(f"Words from index {start_index} to {end_index} in the vocabulary:")
for word, index in vocab:
    print(f"Word: {word}, Vector: {w2v_model.wv[word]}")

Words from index 100 to 200 in the vocabulary:
Word: think, Vector: [-0.00278173 -0.03148978 -0.19988644  0.24513172 -0.20290688 -0.5849748
  0.51498604  0.32279477 -0.6655291  -0.9117937   0.4155266  -0.26454088
 -0.16799182  0.00340257  0.59641427  0.47713745  0.01368268 -0.48095816
 -0.9784495  -0.5673754  -0.44074932  0.50504565  0.3420833  -0.03573836
  0.36454445 -0.1197224  -0.1768053   0.27533805 -0.14537884  0.23030758
  0.1494056   0.48066872  0.52023166  0.37291282 -0.5548476   0.85057795
  0.15305845 -0.02667257 -0.16417648 -0.5597407   0.61683404  0.06247215
 -0.06193737 -0.35880214  0.8189808   0.21891876  0.451265   -0.17744155
  0.10666299 -0.05336195]
Word: object, Vector: [-0.42321137 -0.03990309 -0.4447934   0.10641153 -0.21911657 -0.7371813
  0.32595757  0.47594756 -0.8717318  -0.3649765   0.331148   -0.3325633
  0.19038838  0.43070376 -0.12055181  0.2773573   0.10734705 -0.18791732
 -0.53755146 -0.6667103  -0.4328467   0.09772828  0.7153106   0.07365722
  0.6353741

In [11]:
# Implement Word2Vec Search Function
def word2vec_search(query, num_results=5):
    query_words = query.split()
    valid_words = [word for word in query_words if word in w2v_model.wv.key_to_index]
    if not valid_words:
        print("None of the query words are present in the Word2Vec model's vocabulary.")
        return []
    similarities = [w2v_model.wv.n_similarity(valid_words, word_tokenize(entry['Processed Text'].lower())) for entry in database]
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:num_results]
    return [database[i] for i in top_indices]

In [12]:
# First Variation LLM Search with FlagEmbedding
flag_model = FlagModel('BAAI/bge-large-en-v1.5',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:")

def flag_embedding_search(query, num_results=5):
    embeddings1 = flag_model.encode([entry['Processed Text'] for entry in database])
    queries = flag_model.encode_queries([query])
    similarities = embeddings1 @ queries.T
    top_indices = similarities.argmax(axis=0)[:num_results]
    return [database[i] for i in top_indices]

C:\Users\Patrick\anaconda3\envs\urbandesignenv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Patrick\.cache\huggingface\hub\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
# Second Variation LLM Search with Jina Embeddings
def jina_embedding_search(query, num_results=5):
    cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
    model2 = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
    embeddings2 = model2.encode([entry['Processed Text'] for entry in database])
    query_vec = model2.encode([query])
    similarities2 = cos_sim(embeddings2, query_vec)
    top_indices2 = similarities2.argmax(axis=0)[:num_results]
    return [database[i] for i in top_indices2]

In [13]:
#Testing
query = "shopping curating"
num_results = 5
print("Word2Vec Search Results:", word2vec_search(query, num_results))

None of the query words are present in the Word2Vec model's vocabulary.
Word2Vec Search Results: []


In [14]:
#Testing
query = "shopping curating"
num_results = 5
print("Flag Embedding Search Results:", flag_embedding_search(query, num_results))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings:   1%|▊                                                         | 1/70 [03:42<4:16:14, 222.82s/it]


KeyboardInterrupt: 

In [ ]:
#Testing
query = "shopping curating"
num_results = 5
print("Jina Embedding Search Results:", jina_embedding_search(query, num_results))